In [451]:
# Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [452]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

1) host_response_time

1.	host_response_time
•	host_acceptance_rate
•	host_is_superhost
•	number_of_reviews

In [453]:
print(Naples["host_response_time"].unique())

['Sin especificar' 'within a few hours' 'within an hour'
 'a few days or more' 'within a day']


In [454]:
# Convertir host_response_time a dicotomica
Naples["host_response_time"] = Naples["host_response_time"].replace({
    "within an hour": "Rápido",
    "within a few hours": "Rápido",
    "within a day": "Lento",
    "a few days or more": "Lento",
    "Sin especificar": "Lento"
})

# Verificar que la conversión fue correcta
print(Naples["host_response_time"].value_counts())

host_response_time
Rápido    7972
Lento     3055
Name: count, dtype: int64


In [455]:
print(Naples["host_is_superhost"].unique())

['f' 'Sin especificar' 't']


In [456]:
Naples["host_is_superhost"] = Naples["host_is_superhost"].replace({"f": 0, "t": 1, "Sin especificar": 2})

C:\Users\Bondleton\AppData\Local\Temp\ipykernel_28192\1419231691.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Naples["host_is_superhost"] = Naples["host_is_superhost"].replace({"f": 0, "t": 1, "Sin especificar": 2})


In [457]:
print(Naples["host_is_superhost"].unique())

[0 2 1]


In [458]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['host_acceptance_rate', 'host_is_superhost', 'number_of_reviews']]
Var_Dep = Naples['host_response_time']

In [459]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [460]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['Lento', 'Rápido', 'Rápido', ..., 'Rápido', 'Rápido', 'Rápido'],
      dtype=object)

In [461]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[ 391  513]
 [ 160 2245]]


In [462]:
# calculo precision del modelo 
# rapido o lento - pos_label
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Rápido")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8139956490210297


In [463]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7966152916288909


In [464]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Lento")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.4325221238938053


2) host_is_superhost

2.	host_is_superhost
•	// No lo use host_response_rate (todo es 1)
•	host_acceptance_rate
•	host_total_listings_count
•	host_response_time

In [465]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [471]:
print(Naples["host_response_time"].unique())

['Sin especificar' 'within a few hours' 'within an hour'
 'a few days or more' 'within a day']


In [467]:
# Convertir host_is_superhost a dicotomica
Naples["host_is_superhost"] = Naples["host_is_superhost"].replace({
    "Sin especificar": "No Superhost",
    "f": "No Superhost",
    "t": "Superhost"
})

# Verificar que la conversión fue correcta
print(Naples["host_is_superhost"].value_counts())

host_is_superhost
No Superhost    8202
Superhost       2825
Name: count, dtype: int64


In [472]:
# Convertir host_response_time a dicotomica
Naples["host_response_time"] = Naples["host_response_time"].replace({
    "within an hour": 1,
    "within a few hours": 2,
    "within a day": 3,
    "a few days or more": 4,
    "Sin especificar": 5
})

# Verificar que la conversión fue correcta
print(Naples["host_response_time"].value_counts())

host_response_time
1    7330
5    2468
2     642
3     393
4     194
Name: count, dtype: int64


C:\Users\Bondleton\AppData\Local\Temp\ipykernel_28192\2485131506.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Naples["host_response_time"] = Naples["host_response_time"].replace({


In [473]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['host_response_time', 'host_acceptance_rate', 'host_total_listings_count']]
Var_Dep = Naples['host_is_superhost']

In [474]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [475]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['No Superhost', 'No Superhost', 'No Superhost', ...,
       'No Superhost', 'No Superhost', 'No Superhost'], dtype=object)

In [476]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[2440    0]
 [ 869    0]]


In [477]:
# calculo precision del modelo 
# super o no super - pos_label
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Superhost")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.0


c:\Users\Bondleton\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [478]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7373828951344817


In [479]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="No Superhost")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


3) beds

3.	beds
•	Accommodates
•	Bedrooms
•	Room_type

In [480]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [487]:
print(Naples["bedrooms"].unique())

[1.  3.  1.4 2.  1.3 0. ]


In [482]:
# Convertir host_is_superhost a dicotomica
Naples["beds"] = Naples["beds"].replace({
    0: "Sin cama",
    6: "Con cama",
    5: "Con cama",
    4: "Con cama",
    3: "Con cama",
    2.2: "Con cama",
    2.1: "Con cama",
    2: "Con cama",  
    1: "Con cama"
})

# Verificar que la conversión fue correcta
print(Naples["beds"].value_counts())

beds
Con cama    10778
Sin cama      249
Name: count, dtype: int64


In [488]:
print(Naples["room_type"].unique())

['Private room' 'Entire home/apt' 'Hotel room' 'Shared room']


In [489]:
# Convertir room_type a binario (Privado vs. Compartido)
Naples["room_type"] = Naples["room_type"].replace({
    "Entire home/apt": 1,  # Privado
    "Private room": 2,      # Privado
    "Shared room": 3,       # Compartido
    "Hotel room": 4         # Compartido
})

# Confirmar conversión
print(Naples["room_type"].value_counts())  

room_type
1    7195
2    3621
4     128
3      83
Name: count, dtype: int64


C:\Users\Bondleton\AppData\Local\Temp\ipykernel_28192\12446977.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Naples["room_type"] = Naples["room_type"].replace({


In [490]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['accommodates', 'bedrooms', 'room_type']]
Var_Dep = Naples['beds']

In [491]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [492]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['Con cama', 'Con cama', 'Con cama', ..., 'Con cama', 'Con cama',
       'Con cama'], dtype=object)

In [493]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[3237    0]
 [  72    0]]


In [494]:
# calculo precision del modelo 
# con cama o sin cama - pos_label
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Con cama")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9782411604714415


In [495]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9782411604714415


In [496]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Sin cama")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


4) host_has_profile_pic

4.	host_has_profile_pic
•	host_listings_count
•	host_verifications
•	Host_is_superhost

In [541]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [542]:
print(Naples["host_has_profile_pic"].unique())

['t' 'f']


In [ ]:
# # Convertir host_has_profile_pic a dicotomica
# Naples["host_has_profile_pic"] = Naples["host_has_profile_pic"].replace({
#     "t": "Con foto",
#     "f": "Sin foto",
# })

# # Verificar que la conversión fue correcta
# print(Naples["host_has_profile_pic"].value_counts())

host_has_profile_pic
Con foto    9997
Sin foto    1030
Name: count, dtype: int64


In [543]:
# Convertir host_verifications a dicotomica
Naples["host_verifications"] = Naples["host_verifications"].replace({
    "['email', 'phone']": 1, # Verificado = 1
    "['email', 'phone', 'work_email']": 2,
    "['phone']": 3,
    "['email']": 4,
    "['phone', 'work_email']": 5,
    "[]": "6", # No verificado = 0
})

# Verificar que la conversión fue correcta
print(Naples["host_verifications"].value_counts())

host_verifications
1    9342
3    1193
2     468
4      11
6       7
5       6
Name: count, dtype: int64


In [544]:
# Convertir host_is_superhost a dicotomica
Naples["host_is_superhost"] = Naples["host_is_superhost"].replace({
    "Sin especificar": "0",
    "f": "0",
    "t": "1"
})

# Verificar que la conversión fue correcta
print(Naples["host_is_superhost"].value_counts())

host_is_superhost
0    8202
1    2825
Name: count, dtype: int64


In [545]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['host_listings_count', 'host_verifications', 'host_is_superhost']]
Var_Dep = Naples['host_has_profile_pic']

In [546]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [547]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [548]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[   2  324]
 [   3 2980]]


In [550]:
# calculo precision del modelo 

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9019370460048426


In [551]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9011786038077969


In [553]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.006134969325153374


5) host_identity_verified

5.	host_identity_verified
•	Host_is_superhost
•	review_scores_communication
•	review_scores_rating

In [561]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [562]:
print(Naples["host_is_superhost"].unique())

['f' 'Sin especificar' 't']


In [563]:
# Convertir host_is_superhost a dicotomica
Naples["host_is_superhost"] = Naples["host_is_superhost"].replace({
    "Sin especificar": "3",
    "f": "2",
    "t": "1"
})

# Verificar que la conversión fue correcta
print(Naples["host_is_superhost"].value_counts())

host_is_superhost
2    7814
1    2825
3     388
Name: count, dtype: int64


In [564]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['host_is_superhost', 'review_scores_communication', 'review_scores_rating']]
Var_Dep = Naples['host_identity_verified']

In [565]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [566]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [567]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[   0  238]
 [   0 3071]]


In [568]:
# calculo precision del modelo 

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9280749471139317


In [569]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9280749471139317


In [570]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


6) room_type

6.	room_type
•	property_type
•	accommodates
•	price

In [571]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [576]:
print(Naples["price"].unique())

[ 71.   76.   68.   58.   60.   42.   93.3 117.6  90.  100.   91.   35.
  99.  103.   75.   62.   70.   80.  105.  140.   55.  124.   85.   53.
  82.  110.   46.   40.  159.  120.  109.  126.   20.  102.   89.  183.
  94.   83.   61.   63.   57.   78.  127.   65.   50.   74.  160.  187.
 145.   48.   95.  164.   43.  113.   73.   86.  139.   64.  175.  144.
 119.  148.  190.   72.   30.  123.  135.   92.   38.  153.   47.   67.
  45.   93.  114.   97.   66.   59.   96.  150.   28.   31.  118.  104.
 193.   39.  156.  106.   22.  180.   79.  128.  151.   81.   84.   23.
 107.  162.  101.  147.  167.  132.  115.  166.   88.   27.  174.  130.
  69.  170.   33.  178.   52.   77.  161.   54.  158.   87.   37.   98.
 125.  122.  186.  169.  179.  108.  117.   56.  195.  177.  129.  116.
  36.  134.  111.  168.   13.  194.   18.  112.  121.   51.   26.  146.
  34.  184.  141.  155.   49.  131.  137.  163.  173.   32.  149.  133.
  29.  136.   44.  191.  152.   21.  142.  154.  189.  188.  143

In [573]:
# Convertir room_type a binario (Privado vs. Compartido)
Naples["room_type"] = Naples["room_type"].replace({
    "Entire home/apt": "Privado",  # Privado
    "Private room": "Privado",      # Privado
    "Shared room": "Compartido",       # Compartido
    "Hotel room": "Compartido"        # Compartido
})

# Confirmar conversión
print(Naples["room_type"].value_counts())  

room_type
Privado       10816
Compartido      211
Name: count, dtype: int64


In [577]:
# Crear diccionario con valores numéricos para cada tipo de propiedad
Naples["property_type"] = Naples["property_type"].replace({
    'Private room in bed and breakfast': 1,
    'Private room in condo': 2,
    'Entire rental unit': 3,
    'Private room in rental unit': 4,
    'Entire loft': 5,
    'Entire cottage': 6,
    'Entire home': 7,
    'Room in bed and breakfast': 8,
    'Entire condo': 9,
    'Shared room in hostel': 10,
    'Room in boutique hotel': 11,
    'Entire vacation home': 12,
    'Entire villa': 13,
    'Private room in home': 14,
    'Entire serviced apartment': 15,
    'Entire townhouse': 16,
    'Private room in hostel': 17,
    'Private room': 18,
    'Entire bed and breakfast': 19,
    'Private room in villa': 20,
    'Entire guesthouse': 21,
    'Room in hostel': 22,
    'Boat': 23,
    'Tiny home': 24,
    'Dome': 25,
    'Room in hotel': 26,
    'Shared room in bed and breakfast': 27,
    'Entire guest suite': 28,
    'Room in serviced apartment': 29,
    'Private room in vacation home': 30,
    'Shared room in rental unit': 31,
    'Shared room in condo': 32,
    'Private room in serviced apartment': 33,
    'Entire place': 34,
    'Private room in guest suite': 35,
    'Private room in tiny home': 36,
    'Private room in townhouse': 37,
    'Private room in boat': 38,
    'Private room in loft': 39,
    'Earthen home': 40,
    'Dammuso': 41,
    'Shared room in home': 42,
    'Houseboat': 43,
    'Private room in guesthouse': 44,
    'Private room in casa particular': 45,
    'Camper/RV': 46,
    'Castle': 47,
    'Casa particular': 48,
    'Entire home/apt': 49,
    'Farm stay': 50,
    'Private room in farm stay': 51,
    'Room in aparthotel': 52,
    'Tent': 53
})

# Confirmar conversión
print(Naples["property_type"].value_counts())  

property_type
3     4095
1     1690
9     1366
4     1093
7      921
12     451
2      311
14     142
5      110
8      102
24     102
11      99
30      66
15      58
33      57
26      54
31      42
44      39
35      28
28      23
27      20
13      17
29      17
10      14
23      13
18      11
48      10
45       9
21       9
39       7
36       6
42       4
37       4
17       4
51       3
20       3
32       3
52       3
22       2
43       2
38       2
34       2
16       2
19       2
41       1
40       1
6        1
46       1
47       1
49       1
50       1
25       1
53       1
Name: count, dtype: int64


C:\Users\Bondleton\AppData\Local\Temp\ipykernel_28192\3781815957.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Naples["property_type"] = Naples["property_type"].replace({


In [578]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['property_type', 'accommodates', 'price']]
Var_Dep = Naples['room_type']

In [579]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [580]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['Privado', 'Privado', 'Privado', ..., 'Privado', 'Privado',
       'Privado'], dtype=object)

In [581]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[   0   65]
 [   0 3244]]


In [582]:
# calculo precision del modelo 
# con cama o sin cama - pos_label
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Privado")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9803566032033847


In [583]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9803566032033847


In [584]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Compartido")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


7) has_availability

7.	has_availability
•	price
•	availability_365
•	number_of_reviews

In [585]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [591]:
print(Naples["has_availability"].unique()) 

['Disponible' 'No disponible']


In [587]:
# Convertir
Naples["has_availability"] = Naples["has_availability"].replace({
    "t": "Disponible",
    "f": "No disponible",
    "Sin especificar": "No disponible",
})

# Verificar que la conversión fue correcta
print(Naples["has_availability"].value_counts())

has_availability
Disponible       10994
No disponible       33
Name: count, dtype: int64


In [592]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['price', 'availability_365', 'number_of_reviews']]
Var_Dep = Naples['has_availability']

In [593]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [594]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['Disponible', 'Disponible', 'Disponible', ..., 'Disponible',
       'Disponible', 'Disponible'], dtype=object)

In [595]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[3304    0]
 [   5    0]]


In [596]:
# calculo precision del modelo 

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Disponible")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9984889694771835


In [597]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9984889694771835


In [598]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="No disponible")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


8) instant_bookable

8.	instant_bookable
•	maximum_nights
•	minimum_nights
•	host_response_time

In [599]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [602]:
print(Naples["host_response_time"].unique())

['Sin especificar' 'within a few hours' 'within an hour'
 'a few days or more' 'within a day']


In [603]:
# Convertir host_response_time a dicotomica
Naples["host_response_time"] = Naples["host_response_time"].replace({
    "within an hour": 1,
    "within a few hours": 2,
    "within a day": 3,
    "a few days or more": 4,
    "Sin especificar": 5
})

# Verificar que la conversión fue correcta
print(Naples["host_response_time"].value_counts())

host_response_time
1    7330
5    2468
2     642
3     393
4     194
Name: count, dtype: int64


C:\Users\Bondleton\AppData\Local\Temp\ipykernel_28192\2485131506.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Naples["host_response_time"] = Naples["host_response_time"].replace({


In [604]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['maximum_nights', 'minimum_nights', 'host_response_time']]
Var_Dep = Naples['instant_bookable']

In [605]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [606]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 't', 't', ..., 't', 't', 't'], dtype=object)

In [607]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[ 478  950]
 [ 342 1539]]


In [608]:
# calculo precision del modelo 
# rapido o lento - pos_label
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6183206106870229


In [609]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6095497129042007


In [610]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.33473389355742295


9) bedrooms

9.	bedrooms
•	room_type
•	bathrooms
•	beds

In [ ]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [ ]:
print(Naples["beds"].unique())

[1.  2.  3.  2.2 2.1 6.  4.  5.  0. ]


In [612]:
# Convertir bedrooms a dicotomica
Naples["bedrooms"] = Naples["bedrooms"].replace({
    0: "Sin cuarto",
    1: "Con cuarto",
    1.3: "Con cuarto",
    1.4: "Con cuarto",
    2: "Con cuarto",
    3: "Con cuarto"
})

# Verificar que la conversión fue correcta
print(Naples["bedrooms"].value_counts())

bedrooms
Con cuarto    10834
Sin cuarto      193
Name: count, dtype: int64


In [614]:
# Convertir room_type a binario (Privado vs. Compartido)
Naples["room_type"] = Naples["room_type"].replace({
    "Entire home/apt": 1,  # Privado
    "Private room": 2,      # Privado
    "Shared room": 3,       # Compartido
    "Hotel room": 4         # Compartido
})

# Confirmar conversión
print(Naples["room_type"].value_counts())  

room_type
1    7195
2    3621
4     128
3      83
Name: count, dtype: int64


C:\Users\Bondleton\AppData\Local\Temp\ipykernel_28192\12446977.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Naples["room_type"] = Naples["room_type"].replace({


In [617]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['room_type', 'bathrooms', 'beds']]
Var_Dep = Naples['bedrooms']

In [618]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [619]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['Con cuarto', 'Con cuarto', 'Con cuarto', ..., 'Con cuarto',
       'Con cuarto', 'Con cuarto'], dtype=object)

In [620]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[3245    0]
 [  64    0]]


In [621]:
# calculo precision del modelo 
# con cuarto o sin cuarto - pos_label
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Con cuarto")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.980658809307948


In [622]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.980658809307948


In [ ]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Sin cuarto")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


10) minimum_nights

10.	minimum_nights
•	price
•	room_type
•	availability_365

In [624]:
# Cargar archivo desde equipo
Naples = pd.read_csv("Datos_limpios_Naples.csv")
Naples = Naples.drop(['Unnamed: 0'], axis=1)
# Naples

In [627]:
print(Naples["room_type"].unique())

['Private room' 'Entire home/apt' 'Hotel room' 'Shared room']


In [626]:
# Convertir minimum_nights a dicotomica
Naples["minimum_nights"] = Naples["minimum_nights"].replace({
    1: "Noche",
    1.5: "Noche",
    2: "Noches",
    3: "Noches"
})

# Verificar que la conversión fue correcta
print(Naples["minimum_nights"].value_counts())

minimum_nights
Noche     6072
Noches    4955
Name: count, dtype: int64


In [628]:
# Convertir room_type a binario (Privado vs. Compartido)
Naples["room_type"] = Naples["room_type"].replace({
    "Entire home/apt": 1,  # Privado
    "Private room": 2,      # Privado
    "Shared room": 3,       # Compartido
    "Hotel room": 4         # Compartido
})

# Confirmar conversión
print(Naples["room_type"].value_counts())  

room_type
1    7195
2    3621
4     128
3      83
Name: count, dtype: int64


C:\Users\Bondleton\AppData\Local\Temp\ipykernel_28192\12446977.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Naples["room_type"] = Naples["room_type"].replace({


In [630]:
# Declaramos las variables dependientes e independientes para la regresion logistica
Vars_Indep = Naples[['price', 'room_type', 'availability_365']]
Var_Dep = Naples['minimum_nights']

In [631]:
# Redefinimos las variables, arreglo matricial
X = Vars_Indep
y = Var_Dep

# Conjunto de entrenamienso se ingresa y da un salida con respecto 
# Dividimos  el conjunto de datos en la parte de entrenamiento (70%) y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Escalar todos los datos
escalar = StandardScaler()

# Para realizar el escalamiento de las variables x  tanto de entrenamiento como de prueba utilizamos 
# La y es solo do, y las dependeintes si se pueden escalar
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [632]:
# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo
algoritmo.fit(X_train, y_train)

# Realizamos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred

array(['Noche', 'Noche', 'Noches', ..., 'Noche', 'Noches', 'Noches'],
      dtype=object)

In [633]:
# Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# diagonal: 
# diagonal:

Matriz de Confusión:
[[ 926  927]
 [ 368 1088]]


In [634]:
# calculo precision del modelo 
# con cuarto o sin cuarto - pos_label
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Noche")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.7156105100463679


In [635]:
# Calculo de la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6086430945905107


In [637]:
# Calculo de la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Noches")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7472527472527473
